In [3]:
import cv2
import numpy as np
from tqdm import tqdm
import os
from PIL import Image
from matplotlib import pyplot as plt
from moviepy.video.io.VideoFileClip import VideoFileClip
import glob

In [4]:
os.chdir("C:/Users/user/Desktop/Lab/Kaist/code")

In [5]:
class videofix:
  # 영상 자르기
  def cutVideo(start_sec,end_sec,input_filepath,output_filepath):
    ## 비디오를 가져와서 내가 원하는 부분에 대한 seconds 설정
    clip = VideoFileClip(input_filepath).subclip(start_sec,end_sec)
    ## 새로운 비디오 파일 저장
    clip.write_videofile(output_filepath)


  # 영상 프레임별로 사진 저장
  def saveImg(input_filepath):
    clip = VideoFileClip(input_filepath)
    ## 프레임을 추출해서 이미지 저장
    os.mkdir("./framebyimg")
    for i, frame in enumerate(clip.iter_frames()):
      image = Image.fromarray(frame)
      image.save("./framebyimg/frame_{}.jpg".format(i))


  # 사진 구도 조정 후 영상으로 변환
  def removeImg(save_videoname):
    ## 동영상 순서별로 사진 순서 조정
    os.chdir("./framebyimg")
    imgs = os.listdir()
    imgs_ = {}
    for img in imgs:
      if img.endswith('.jpg'):
        img_idx = img.split('_')[-1].split('.')[0]
        imgs_[int(img_idx)] = img
    res = list(map(lambda x: x[1], sorted(imgs_.items(), key=lambda x: x[0])))

    ## Optical Flow를 활용해 사진구도 조정
    ## Load the images
    imgs = res
    imgs_aligned = []

    ## Read the image
    prev_image = cv2.imread(imgs[0])
    prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)

    ## 디텍터 정의
    feature_detector = cv2.GFTTDetector_create()

    for i in tqdm(range(1,len(imgs))):
      ###첫 번째 프레임에서 형상 탐지
      prev_features = feature_detector.detect(prev_gray)
      prev_features = np.array([x.pt for x in prev_features], dtype = np.float32)

      ### 다음이미지 불러오기
      curr_image = cv2.imread(imgs[i])
      curr_gray = cv2.cvtColor(curr_image, cv2.COLOR_BGR2GRAY)

      ### 광학 흐름 계산
      next_features, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_features, None)

      ### 카메라의 움직임을 계산
      motion, _ = cv2.estimateAffine2D(prev_features, next_features)[:2]

      ### 이미지 흔들림 보정
      corrected_image = cv2.warpAffine(curr_image, motion, (curr_image.shape[1], curr_image.shape[0]), flags = cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)

      imgs_aligned.append(corrected_image)
      
    ## 영상 저장
    out = cv2.VideoWriter('../{}.mp4'.format(save_videoname),cv2.VideoWriter_fourcc(*'FMP4'), 40, (3840, 2160))

    for i in range(len(imgs_aligned)):
        out.write(imgs_aligned[i])
    out.release()
    [os.remove(f) for f in glob.glob("./*.jpg")]
    os.chdir("../")
    os.rmdir("./framebyimg")




In [ ]:
# 영상자르기
input_filepath = "C:/Users/user/Desktop/Lab/Kaist/yolo_video/Songdo.mp4"
output_filepath = "C:/Users/user/Desktop/Lab/Kaist/code/CUT_songdo.mp4"

videofix.cutVideo(0,190,input_filepath,output_filepath)

In [4]:
# 영상자르기
input_filepath = "G:/.shortcut-targets-by-id/1oV6srM2tpa3LfD_wLXITXyJLEn8uyQDq/박민주교수님/Songdo/DJI_0026.mp4"
output_filepath = "C:/Users/user/Desktop/Lab/Kaist/code/CUT_DJI_0026.mp4"

videofix.cutVideo(0,190,input_filepath,output_filepath)

Moviepy - Building video C:/Users/user/Desktop/Lab/Kaist/code/CUT_DJI_0026.mp4.
Moviepy - Writing video C:/Users/user/Desktop/Lab/Kaist/code/CUT_DJI_0026.mp4



Moviepy - Done !
Moviepy - video ready C:/Users/user/Desktop/Lab/Kaist/code/CUT_DJI_0026.mp4


In [7]:
# 프레임별 이미지 생성
output_filepath = "C:/Users/user/Desktop/Lab/Kaist/code/CUT_DJI_0026.mp4"
videofix.saveImg(output_filepath)

KeyboardInterrupt: 

In [15]:
# 이미지 조정 후 동영상 재저장
videofix.removeImg("remake_D26")

100%|██████████| 5694/5694 [30:38<00:00,  3.10it/s]  
